# 1. River part

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# ==============================================
# Heavy Metals Dataset Analysis: Mackay vs Normanby
# ==============================================

# 2️⃣ Load the Dataset
file_path = "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
df = pd.read_csv(file_path)

# 3️⃣ Basic Inspection
print("=== Dataset Overview ===")
print(df.info())
print("\n=== First 20 Rows ===")
print(df.head())

# 4️⃣ Check for Missing Data
print("\n=== Missing Values ===")
print(df.isnull().sum())

# 5️⃣ Basic Descriptive Statistics
print("\n=== Descriptive Statistics ===")
print(df.describe())

# 6️⃣ Plot Distribution of Each Metal
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols].hist(figsize=(12, 8), bins=20)
plt.suptitle("Distribution of Heavy Metal Concentrations", fontsize=14)
plt.tight_layout()
plt.savefig("fig_Distribution of Heavy Metals.png")
plt.show()
  
# 7️⃣ Compare Mackay vs Normanby (if 'Region' column exists)
if 'Region' in df.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=df, x='Region', y=numeric_cols[0])  # First numeric column as example
    plt.title(f"{numeric_cols[0]} Concentration Comparison by Region")
    plt.savefig("fig_Concentration Comparison by region.png")
    plt.show()

    # Loop through all metals to compare between sites
    for col in numeric_cols:
        plt.figure(figsize=(8, 5))
        sns.boxplot(data=df, x='Region', y=col)
        plt.title(f"{col} Concentration by Region")
        plt.savefig("fig_Concentration by region.png")
        plt.show()

# 8️⃣ Correlation Matrix Between Metals
plt.figure(figsize=(10, 6))
sns.heatmap(df[numeric_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix of Heavy Metal Concentrations")
plt.savefig("fig_Correlation matrix of Heavy Metals Concentration.png")
plt.show()


# 9️⃣ Group Statistics (Mean Concentration per Region)
if 'Region' in df.columns:
    site_summary = df.groupby('Region')[numeric_cols].mean().round(2)
    print("\n=== Mean Metal Concentration per Region ===")
    plt.savefig("fig_Metal concentration per region.png")
    print(site_summary)

   
# 🔟 Export Summary to CSV
output_file = "Metal_Concentration_Summary.csv"
site_summary.to_csv(output_file)
plt.savefig("fig_Metal concentration summary.png")
print(f"\nSummary saved to: {output_file}")



In [19]:
# ==============================================
# Time-Series Analysis of Heavy Metal Concentrations
# Mackay vs Normanby, QLD
# ==============================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Load the dataset
file_path = "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
df = pd.read_csv(file_path)

# 2️⃣ Inspect structure
print(df.head())
print(df.info())

# 3️⃣ Ensure the date column is parsed correctly
# 🔹 Change 'Date' to the actual date/time column name in your file
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# 4️⃣ Sort by date
df = df.sort_values('Date')

# 5️⃣ List numeric columns (heavy metals)
metal_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# 6️⃣ Check for unique sites
print("\nRegion:", df['Region'].unique())

# 7️⃣ Plot time series for each metal by region
for metal in metal_cols:
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y=metal, hue='Region', marker='o')
    plt.title(f"{metal} Concentration Over Time by Region")
    plt.xlabel("Date")
    plt.ylabel(f"{metal} (mg/L or µg/L)")
    plt.legend(title="Region")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("fig_Metal concentration summary.png")
    plt.show()

# 8️⃣ Compute rolling mean (trend smoothing)
window_size = 3  # e.g., 3-day or 3-sample smoothing
for metal in metal_cols:
    df[f'{metal}_rolling'] = df.groupby('Region')[metal].transform(lambda x: x.rolling(window_size, min_periods=1).mean())

# 9️⃣ Plot smoothed trends
for metal in metal_cols:
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y=f'{metal}_rolling', hue='Region')
    plt.title(f"Smoothed Trend of {metal} Concentration ({window_size}-Point Rolling Mean)")
    plt.xlabel("Date")
    plt.ylabel(f"{metal} (Rolling Avg)")
    plt.legend(title="Region")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("fig_Smoothed Trend of Metal Concentration.png")
    plt.show()

# 🔟 Region-wise summary over time (mean per site per month)
df['Month'] = df['Date'].dt.to_period('M')
monthly_summary = df.groupby(['Region', 'Month'])[metal_cols].mean().reset_index()

print("\n=== Monthly Average Concentrations by Region ===")
print(monthly_summary.head())
plt.savefig("fig_Region-wise summary.png")
plt.show()

# 11️⃣ Optional: export the monthly summary
monthly_summary.to_csv("Monthly_HeavyMetals_Summary.csv", index=False)
print("Saved summary to 'Monthly_HeavyMetals_Summary.csv'")
plt.savefig("fig_Optional Export Montly summary.png")
plt.savefig("fig_Optional Export Monthly summary.png")
plt.show()


In [3]:
# ==============================================
# Heavy Metal Concentration Analysis by Region/Site
# ==============================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Load the dataset
file_path = "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
df = pd.read_csv(file_path)

# 2️⃣ Inspect the structure
print("\n=== Dataset Preview ===")
print(df.head())
print("\n=== Column Info ===")
print(df.info())

# 3️⃣ Identify key columns
# Update 'Site' to match your actual column name for regions
region_col = 'Site'

# Automatically select numeric columns (assumed to be heavy metal concentrations)
metal_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

print("\nDetected Heavy Metal Columns:", metal_cols)
print("\nDetected Region Column:", region_col)

# 4️⃣ Check for missing values
print("\n=== Missing Values by Column ===")
print(df.isnull().sum())

# 5️⃣ Descriptive statistics per region
region_summary = df.groupby(region_col)[metal_cols].describe().round(2)
print("\n=== Summary Statistics by Region ===")
print(region_summary)

# 6️⃣ Mean concentration of each metal per region
mean_concentrations = df.groupby(region_col)[metal_cols].mean().round(3)
print("\n=== Mean Heavy Metal Concentrations per Region ===")
print(mean_concentrations)

# 7️⃣ Plot: Boxplots for each metal by region
for metal in metal_cols:
    plt.figure(figsize=(8, 5))
    sns.boxplot(data=df, x=region_col, y=metal, palette="Set2")
    plt.title(f"{metal} Concentration by Region")
    plt.xlabel("Region/Site")
    plt.ylabel(f"{metal} (mg/L or µg/L)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# 8️⃣ Plot: Mean concentrations (bar chart)
mean_concentrations.T.plot(kind='bar', figsize=(10, 6))
plt.title("Average Heavy Metal Concentrations by Region")
plt.xlabel("Heavy Metal")
plt.ylabel("Mean Concentration")
plt.legend(title="Region/Site")
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# 9️⃣ Heatmap of correlations (between metals)
plt.figure(figsize=(10, 6))
sns.heatmap(df[metal_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix of Heavy Metals")
plt.tight_layout()
plt.show()

# 🔟 Export summary to CSV
output_file = "Region_HeavyMetals_Summary.csv"
mean_concentrations.to_csv(output_file)
print(f"\nSummary exported to '{output_file}'")


In [4]:
# =====================================================
# Environmental Simulation and Visualisation of Heavy Metals by Site
# =====================================================

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

# 1️⃣ Load the dataset
file_path = "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
df = pd.read_csv(file_path)

# 2️⃣ Inspect dataset
print(df.head())
print(df.info())

# 3️⃣ Parse key columns
# ⚠️ Adjust column names to match your CSV
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
region_col = 'Site'
lat_col = 'Latitude' if 'Latitude' in df.columns else None
lon_col = 'Longitude' if 'Longitude' in df.columns else None

# Identify numeric heavy-metal columns
metal_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
print("Heavy Metal Columns:", metal_cols)

# 4️⃣ Descriptive analysis
mean_conc = df.groupby(region_col)[metal_cols].mean().round(3)
print("\n=== Mean Heavy-Metal Concentration by Site ===")
print(mean_conc)

# 5️⃣ Plot: Mean heavy-metal concentration per site
plt.figure(figsize=(10,6))
sns.heatmap(mean_conc, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Mean Heavy-Metal Concentration per Site")
plt.xlabel("Metal Type")
plt.ylabel("Region / Site")
plt.tight_layout()
plt.show()

# 6️⃣ Spatial visualisation (if coordinates exist)
if lat_col and lon_col:
    # Create base map centered on mean coordinates
    m = folium.Map(location=[df[lat_col].mean(), df[lon_col].mean()], zoom_start=6, tiles='Stamen Terrain')

    # Add circle markers proportional to mean metal concentration
    for site, group in df.groupby(region_col):
        lat, lon = group[lat_col].mean(), group[lon_col].mean()
        popup_text = f"<b>{site}</b><br>"
        popup_text += "<br>".join([f"{metal}: {group[metal].mean():.2f}" for metal in metal_cols])
        avg_intensity = group[metal_cols].mean().mean()

        folium.CircleMarker(
            location=[lat, lon],
            radius=6 + avg_intensity * 0.2,   # scale radius by concentration
            popup=popup_text,
            color='red' if avg_intensity > df[metal_cols].mean().mean() else 'green',
            fill=True,
            fill_opacity=0.7
        ).add_to(m)

    m.save("HeavyMetals_Site_Map.html")
    print("\n🌏 Interactive map saved as 'HeavyMetals_Site_Map.html'")

# 7️⃣ Time evolution visualisation (simulation-style)
for metal in metal_cols:
    plt.figure(figsize=(10,5))
    sns.lineplot(data=df, x='Date', y=metal, hue=region_col, marker='o')
    plt.title(f"{metal} Concentration Over Time by Site")
    plt.xlabel("Date")
    plt.ylabel(f"{metal} Concentration (mg/L or µg/L)")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# 8️⃣ Simple environmental simulation proxy
# → Here, we simulate the relative spread (normalised values)
df_sim = df.copy()
for metal in metal_cols:
    df_sim[f"{metal}_norm"] = (df_sim[metal] - df_sim[metal].min()) / (df_sim[metal].max() - df_sim[metal].min())

df_sim['Pollution_Index'] = df_sim[[f"{m}_norm" for m in metal_cols]].mean(axis=1)

plt.figure(figsize=(10,6))
sns.barplot(data=df_sim, x=region_col, y='Pollution_Index', ci=None, palette='RdYlGn_r')
plt.title("Simulated Pollution Intensity by Site")
plt.ylabel("Pollution Index (0–1)")
plt.xlabel("Region / Site")
plt.tight_layout()
plt.show()


In [31]:
# ============================================================
# Extract Mackay & Cape York and assess heavy-metal concentration
# Dataset: "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ---------- 1) Load ----------
fp = "Concentration_2025-10-20_22-50-Metals (Makay_Normanby).csv"
df = pd.read_csv(fp)

# Expecting these columns (from your file preview):
# ['Region','Site Name','Latitude','Longitude','Date Time','Analyte','Value','Unit', ...]
required = ['Region','Date Time','Analyte','Value']
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ---------- 2) Basic cleaning ----------
# Parse datetime
df['Date Time'] = pd.to_datetime(df['Date Time'], errors='coerce')

# Clean Value (handle strings like '<5' or ' < 0.1')
def to_float(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = s.replace('<', '').replace('>', '')  # drop qualifier
    # remove any accidental non-numeric characters except . and -
    s = ''.join(ch for ch in s if (ch.isdigit() or ch in '.-'))
    try:
        return float(s) if s != '' else np.nan
    except:
        return np.nan

df['Value'] = df['Value'].apply(to_float)

# Keep only rows with valid time & value
df = df.dropna(subset=['Date Time','Value'])

# ---------- 3) Filter to Mackay & Cape York ----------
# Tolerant matching (e.g., "Mackay", "Mackay Whitsunday", "Cape York")
regions_to_keep = ['mackay', 'cape york']  # edit if your labels are different
mask = df['Region'].astype(str).str.lower().apply(
    lambda r: any(key in r for key in regions_to_keep)
)
df_sel = df[mask].copy()

if df_sel.empty:
    raise ValueError("No rows matched the regions. Check the exact spellings in the 'Region' column.")

# Normalise Region labels to two buckets for clarity
def normalise_region(r):
    rl = str(r).lower()
    if 'cape york' in rl:
        return 'Cape York'
    if 'mackay' in rl:
        return 'Mackay'
    return r  # fallback

df_sel['Region_norm'] = df_sel['Region'].apply(normalise_region)

# ---------- 4) Quick descriptive stats ----------
by_region_analyte = (
    df_sel
    .groupby(['Region_norm','Analyte'])
    .agg(
        n=('Value','count'),
        mean=('Value','mean'),
        median=('Value','median'),
        p90=('Value', lambda x: np.nanpercentile(x, 90)),
        min=('Value','min'),
        max=('Value','max')
    )
    .round(3)
    .reset_index()
)

print("\n=== Summary by Region & Analyte ===")
print(by_region_analyte.head(20))  # show a sample

# Save summary for reporting
by_region_analyte.to_csv("summary_region_analyte.csv", index=False)
print("Saved: summary_region_analyte.csv")

# ---------- 5) Optional: pick a small set of key analytes to plot ----------
# If you want to plot *all* analytes, leave this as None
key_metals = None
# Example: key_metals = ["Arsenic", "Cadmium", "Chromium", "Copper", "Lead", "Zinc"]

if key_metals is not None:
    df_plot = df_sel[df_sel['Analyte'].isin(key_metals)].copy()
else:
    # take the top few most sampled metals to keep plots quick & legible
    top = (
        df_sel['Analyte']
        .value_counts()
        .head(6)  # adjust how many analytes to visualize
        .index
        .tolist()
    )
    df_plot = df_sel[df_sel['Analyte'].isin(top)].copy()

# ---------- 6) Boxplots: distribution by region ----------
for metal in sorted(df_plot['Analyte'].unique()):
    sub = df_plot[df_plot['Analyte'] == metal]
    plt.figure(figsize=(7,5))
    # Use matplotlib directly to avoid extra dependencies
    data = [sub.loc[sub['Region_norm']=='Mackay','Value'],
            sub.loc[sub['Region_norm']=='Cape York','Value']]
    plt.boxplot(data, labels=['Mackay','Cape York'], showfliers=False)
    plt.title(f"{metal} — Distribution by Region")
    plt.ylabel(f"Concentration ({sub['Unit'].mode().iat[0] if not sub['Unit'].mode().empty else ''})")
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig("fig_Metal Distribut by region.png")
    plt.show()


# ---------- 7) Time series: monthly medians by region ----------
df_plot['Month'] = df_plot['Date Time'].dt.to_period('M').dt.to_timestamp()

monthly = (
    df_plot.groupby(['Month','Region_norm','Analyte'])['Value']
    .median()
    .reset_index()
)

for metal in sorted(monthly['Analyte'].unique()):
    sub = monthly[monthly['Analyte'] == metal].copy()
    # Pivot for easy plotting
    pv = sub.pivot(index='Month', columns='Region_norm', values='Value').sort_index()
    plt.figure(figsize=(9,5))
    plt.plot(pv.index, pv.get('Mackay'), marker='o', label='Mackay')
    plt.plot(pv.index, pv.get('Cape York'), marker='o', label='Cape York')
    plt.title(f"{metal} — Monthly Median Concentration")
    plt.xlabel("Month")
    plt.ylabel("Median Concentration")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend()
    plt.tight_layout()
    plt.savefig("fig_Montly Median concentration.png")
    plt.show()

# ---------- 8) Site map (optional if coordinates available) ----------
has_latlon = {'Latitude','Longitude'}.issubset(df_sel.columns)
if has_latlon:
    # Per-site mean across all analytes (or choose one analyte to focus)
    site_avg = (
        df_sel.groupby(['Region_norm','Site Name','Latitude','Longitude'])['Value']
        .mean().reset_index()
    )
    print("\nTop 10 sites by average concentration (across analytes):")
    print(site_avg.sort_values('Value', ascending=False).head(10))
    # You can export this for GIS mapping:
    site_avg.to_csv("site_average_concentration.csv", index=False)
    print("Saved: site_average_concentration.csv")

print("\nDone ✅")
